In [8]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
%matplotlib inline 
Event_df = pd.read_csv('logs1516.csv')
Event_df.sort_values(by=["Time"], ascending=True)
M_df = pd.read_csv('M_results1516.csv')
S_df = pd.read_csv('S_results1516.csv')

In [9]:
#AverageScore
Ave_score = pd.merge(M_df,S_df,on="Full name")
Ave_score["Ave_score"]= Ave_score.apply(lambda x: (x["M_score"] + x ["S_score"] )/2, axis=1).astype('int') 

In [10]:
#Merge with ScoreTable
Event_df_merge = pd.merge(Event_df,Ave_score,on="Full name")
Event_df_merge =Event_df_merge[["Full name","Ave_score"]].drop_duplicates(['Full name', 'Ave_score'])
Event_df_merge = Event_df_merge.sort_values(by=["Full name"], ascending=True).reset_index( drop = True )
Event_df_merge = Event_df_merge.set_index("Full name")
#Event_df_merge

In [11]:
#Count Clikcs on "R" contents
Clicks = Event_df[Event_df["Event context"].str.contains("_R")&Event_df["Event name"].str.contains("Course module viewed")]
Clicks = Clicks["Event context"].groupby(Clicks["Full name"])
Event_df_merge["R_Clicks"] = Clicks.count()
#Event_df_merge

In [12]:
#Count Clikcs on "L" contents
Clicks = Event_df[Event_df["Event context"].str.contains("_L")&Event_df["Event name"].str.contains("Sco launched")]
Clicks = Clicks["Event context"].groupby(Clicks["Full name"])
Event_df_merge["L_Clicks"] = Clicks.count()
#Event_df_merge

In [13]:
#Count Clikcs on "S" contents
Clicks = Event_df[Event_df["Event context"].str.contains("_S")&Event_df["Event name"].str.contains("Course module viewed")]
Clicks = Clicks["Event context"].groupby(Clicks["Full name"])
Event_df_merge["S_Clicks"] = Clicks.count()
Event_df_merge = Event_df_merge[["Ave_score","R_Clicks","L_Clicks","S_Clicks"]] .reset_index().fillna(0)
Event_df_merge=Event_df_merge.drop("Full name", axis=1)
#Event_df_merge
Event_df_merge.to_csv('PCA.csv')

In [219]:
#Caluculating multiple regression
from sklearn import linear_model
clf = linear_model.LinearRegression()

# Explanatory variable: "R_Clicks" ,"L_Clicks" , and "S_Clicks"  except "Ave_score"
Event = Event_df_merge.drop("Ave_score", axis=1)
X = Event.as_matrix()

# Objectitve variable: "Ave_score" 
Y = Event_df_merge['Ave_score'].as_matrix()

# Create predicitive model
clf.fit(X, Y)
 
#  Regression coefficient 
print(pd.DataFrame({"Name":Event.columns,
                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# Intercept
print(clf.intercept_)

   Coefficients      Name
0     -0.107985  R_Clicks
1      0.007260  L_Clicks
2      0.229464  S_Clicks
62.2043398848
